<h1 align="center">Yleistetyt Lineaariset Mallit 2, Demo 3</h1>

<br>

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

<h3 align="left">Tehtävä 1</h3>

Palataan vielä lineaarisen mallin tapaukseen ja regressioestimaattien luottamusvälien laskemiseen bootstrap-menetelmällä.

a) Käy läpi bootsrap-algoritmit kurssin luentomonisteesta

b) Laske PISA-datasta sovitetun mallin regressioestimaattien $ 95 \% $:n luottamusvälit käyttäen 
- parametritonta bootstrap-menetelmää
- parametrista bootsrap-menetelmää
- estimaattien t-jakaumaoletusta

In [2]:
pisa = pd.read_csv("C:/Users/testi/Desktop/excel_files/pisa_demo02.csv", 
                   index_col=0)

In [3]:
pisa.head(2)

,mpist,sukup,SES,koulusij,koulualue,matem,aidink
107,569.254,poika,0.1322,kaupunki,Etela-Suomi,9,7
1261,631.804,poika,1.1144,kaupunki,Etela-Suomi,8,8


In [4]:
# Luodaan deisgn-matriisi.

mat = pisa["matem"].values

sp = pisa["sukup"].replace({"poika": 0, "tytto": 1}).values

sij = pisa["koulusij"].replace({"kaupunki": 0, "maaseutu": 1}).values

ita = np.where(pisa["koulualue"] == "Ita-Suomi", 1, 0)

lansi = np.where(pisa["koulualue"] == "Lansi-Suomi", 1, 0)

pohjoinen = np.where(pisa["koulualue"] == "Pohjois-Suomi", 1, 0)

intercept = np.ones(200, dtype=int)

X = pd.DataFrame({"intercept": intercept,
                  "mat": mat,
                  "sp": sp,
                  "sij": sij,
                  "ita": ita,
                  "lansi": lansi,
                  "pohjoinen": pohjoinen})

X.index += 1

In [5]:
X.head()

,intercept,mat,sp,sij,ita,lansi,pohjoinen
1,1,9,0,0,0,0,0
2,1,8,0,0,0,0,0
3,1,7,1,0,0,0,0
4,1,8,1,0,0,0,0
5,1,6,1,0,0,1,0


In [6]:
# Selittävien muuttujien arvot (NumPy-taulukkona)
X = X.values

In [7]:
# Vastemuuttujan arvot (NumPy-taulukkona)
y = pisa["mpist"].values

In [8]:
y.shape

(200,)

<br>

**Luottamusvälit *parametrittomalla* bootstrap-menetelmällä**

Jos on syytä epäillä, että mallin virhetermien normaalisuusoletus ei
päde, luottamusvälit voidaan laskea parametrittomalla
bootstrap-menetelmällä, jonka algoritmi on seuraavanlainen:

1. Poimi satunnaisesti indeksit $ i_1 \, ,..., \, i_n $ takaisinpalauttaen joukosta $ \{1, 2,...,n\}. $
2. Poimi bootstrap-otos $ (y_{ij}^*, x_{ij}^*), \, j = 1,...,n. $
3. Laske otoksesta bootstrap-estimaatti $ \hat{\beta}^*. $
4. Toista kohtia 1-3 $ M $ kertaa. Tällöin saat estimaatit $ \hat{\beta}_1^*,...,\hat{\beta}_M^*. $
5. Etsi bootstrap-estimaattien $ \large\frac{\alpha}{2} $ ja $ 1 - \large\frac{\alpha}{2} $ kvantiilit. Nämä antavat luottamusvälit, joiden peittotodennäköisyys on likimäärin $ 1 - \alpha. $

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
def bootstrap(y, X):
    n = len(y)
    s = np.random.choice(np.arange(n), size=n, replace=True)
    model = LinearRegression(fit_intercept=False).fit(X[s], y[s])
    return model.coef_

In [11]:
M = 5000
# Taulukon dimensioiden täytyy olla (7, M), koska jokainen bootstrapping tuottaa 7 kerrointa,
# jotka sijoitetaan sarakvektoreiksi.
bs_arr = np.zeros((X.shape[1], M))

for i in range(M):
    bs_arr[:, i] = bootstrap(y, X)

In [12]:
# Tulostetaan kolme ensimmäistä saraketta. 
# Huomaa, että kukin bootstrapping tuottaa oman sarakevektorinsa.
bs_arr[:, 0:3]

array([[347.41569604, 337.14338264, 302.65675933],
       [ 28.72969278,  29.12785987,  33.50277994],
       [-16.72902347, -13.7770715 , -16.61935165],
       [ 30.93051868,  20.08829295,  25.09677367],
       [-17.28859656, -20.44655521, -11.59356704],
       [-48.43010665, -16.36896679, -20.48163083],
       [-28.82319129, -19.87148732, -10.58868926]])

Luottamusvälit saadaan prosenttipistemenetelmällä (algoritmin kohta 5).

In [13]:
bs_quantiles = np.percentile(bs_arr, q=[2.5, 97.5], axis=1).T

In [14]:
df_quantiles = pd.DataFrame({"lower ci": bs_quantiles[:, 0],
                             "upper ci": bs_quantiles[:, 1]},
                             index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

In [15]:
# Luottamusvälit (parametriton bootstrap)
df_quantiles

,lower ci,upper ci
intercept,283.079443,383.782570
mat,23.554707,36.662792
sp,-34.878846,0.484101
sij,-4.277258,34.681058
ita,-34.252844,27.486721
lansi,-51.966756,-11.585128
pohjoinen,-51.088550,26.384949


<br>

**Luottamusvälit *parametrisella* bootstrap-menetelmällä**

In [16]:
import statsmodels.api as sm

In [17]:
def bs_parametric(y, X):
    n = len(y)
    s = np.random.choice(np.arange(n), size=n, replace=True)
    fit0 = sm.OLS(y, X).fit()
    eps = fit0.resid[s]
    ys = np.matmul(X, fit0.params) + eps
    fit = sm.OLS(ys, X).fit()
    beta = pd.DataFrame({"coeff": fit.params, "std_error": fit.bse}).values
    z = (beta[:, 0] - fit0.params) / beta[:, 1]
    return z

In [18]:
M = 5000
bs_parametric_arr = np.zeros((X.shape[1], M))

for i in range(M):
    bs_parametric_arr[:, i] = bs_parametric(y, X)

In [19]:
bs_parametric_quantiles = np.percentile(bs_parametric_arr, q=[2.5, 97.5], axis=1).T

In [22]:
fit = sm.OLS(y, X).fit()

In [23]:
ci = (fit.params.reshape(-1, 1) - bs_parametric_quantiles * fit.bse.reshape(-1, 1))

In [24]:
df2_quantiles = pd.DataFrame({"lower ci": ci[:, 1], 
                              "upper ci": ci[:, 0]},
                              index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

In [25]:
# Luottamusvälit (parametrinen bootstrap)
df2_quantiles

,lower ci,upper ci
intercept,286.065220,384.042068
mat,23.452197,36.282090
sp,-35.475772,0.511321
sij,-8.545552,38.271583
ita,-32.958028,27.298524
lansi,-52.307278,-9.416044
pohjoinen,-40.654218,17.058619


<br>

**Luottamusvälit estimaattien t-jakaumaoletusta käyttäen**

In [26]:
malli = sm.OLS(y, X).fit()

In [27]:
n = len(y)

In [28]:
p = X.shape[1]

In [29]:
ala = malli.params - scipy.stats.t.ppf(0.975, df=n-p-1) * malli.bse

In [30]:
yla = malli.params + scipy.stats.t.ppf(0.975, df=n-p-1) * malli.bse

In [31]:
luottamusvalit = pd.DataFrame({"alaraja": ala,
                               "ylaraja": yla},
                               index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

In [32]:
# Luottamusvälit (t-jakaumaoletus)
luottamusvalit

,alaraja,ylaraja
intercept,285.943838,383.282019
mat,23.509356,36.423179
sp,-35.152642,0.467311
sij,-8.182413,38.391992
ita,-33.674219,26.516273
lansi,-52.771748,-9.827143
pohjoinen,-40.539174,16.754869
